We adjust the number of input nodes to reflect the number of datapoints in each trainning example.

In [28]:
import struct as st, numpy as np, matplotlib.pyplot as plt
import sys
# from IPython.core import page
# page.page(variable)
# %page -r <variablename>

filename_train = {'images': 'mnist/train/train-images-idx3-ubyte',
                  'labels': 'mnist/train/train-labels-idx1-ubyte'};
filename_test = {'images': 'mnist/test/test-images-idx3-ubyte',
                 'labels': 'mnist/test/test-labels-idx1-ubyte'};

In [29]:
def read_train_set_images(filename): 
    images_file = open(filename, 'rb');
    images_file.seek(0);
    magic = st.unpack('>4B', images_file.read(4));
    print(magic);
    number_images = st.unpack('>I', images_file.read(4))[0];
    print('images_n:' + str(number_images));
    number_rows = st.unpack('>I', images_file.read(4))[0];
    number_columns = st.unpack('>I', images_file.read(4))[0];
    print('Rows: ' + str(number_rows) + 'Columns: ' + str(number_columns));
    n_bytes =  1000 * (number_rows * number_columns);
    train_array_images = 255 - np.asarray(st.unpack('>' + 'B' * n_bytes, 
                                    images_file.read(n_bytes))).reshape((1000, number_rows, number_columns));
    print('Images: ' + str(train_array_images.shape));
    return train_array_images;

def read_train_set_labels(filename):
    labels_file = open(filename, 'rb');
    labels_file.seek(0);
    magic_number = st.unpack('>4B', labels_file.read(4));
    number_items = st.unpack('>I', labels_file.read(4))[0];
    n_bytes = 1000;
    train_array_labels =  9 - np.asarray(st.unpack('>' + 'B' * n_bytes,
                                            labels_file.read(n_bytes)))
    print('Labels: ' + str(train_array_labels.shape));
    return train_array_labels;

In [30]:
def read_test_set_images(filename):
    images_file = open(filename, 'rb');
    images_file.seek(0);
    magic = st.unpack('>4B', images_file.read(4));
    print(magic);
    number_images = st.unpack('>I', images_file.read(4))[0];
    number_rows = st.unpack('>I', images_file.read(4))[0];
    number_columns = st.unpack('>I', images_file.read(4))[0];
    print('Rows: ' + str(number_rows) + 'Columns: ' + str(number_columns));
    n_bytes =  1000 * (number_rows * number_columns);
    test_array_images = 255 - np.asarray(st.unpack('>' + 'B' * n_bytes, 
                                    images_file.read(n_bytes))).reshape((1000, number_rows, number_columns));
    print('Images: ' + str(test_array_images.shape));
    return test_array_images;

def read_test_set_labels(filename):
    labels_file = open(filename, 'rb');
    labels_file.seek(0);
    magic_number = st.unpack('>4B', labels_file.read(4));
    number_items = st.unpack('>I', labels_file.read(4))[0];
    n_bytes = 1000;
    test_array_labels =  9 - np.asarray(st.unpack('>' + 'B' * n_bytes,
                                            labels_file.read(n_bytes)))
    print('Labels: ' + str(test_array_labels.shape));
    return test_array_labels;

In [31]:
#train
#defining train images array
train_images_array = read_train_set_images(filename_train['images']) / 255;

#defining train labels array
train_labels_array = read_train_set_labels(filename_train['labels']);
one_hot_train_labels = np.zeros((len(train_labels_array), 10));
for idx,val in enumerate(train_labels_array):
    one_hot_train_labels[idx][val] = val
train_labels_array = one_hot_train_labels;
print(train_labels_array.shape);

#test
#defining test images array
test_images_array = read_test_set_images(filename_test['images']) / 255;

#defining train labels array
test_labels_array = read_test_set_labels(filename_test['labels']);
one_hot_test_labels = np.zeros((len(test_labels_array), 10));
for idx,val in enumerate(test_labels_array):
    one_hot_test_labels[idx][val] = val
test_labels_array = one_hot_test_labels;
print(test_labels_array.shape);

(0, 0, 8, 3)
images_n:60000
Rows: 28Columns: 28
Images: (1000, 28, 28)
Labels: (1000,)
(1000, 10)
(0, 0, 8, 3)
Rows: 28Columns: 28
Images: (1000, 28, 28)
Labels: (1000,)
(1000, 10)


In [32]:
#flatten example
zeros = np.zeros((4, 5, 6, 7, 8));
zeros = zeros.reshape((*zeros.shape[:1], -1, *zeros.shape[-1:]));
print(zeros.shape[:1]);
# np.reshape(()) can take a -1 as an argument, meaning 'total array
# size divided by product of all other listed dimensions'

(4,)


In [33]:
#flatten train images array;
train_images_array = train_images_array.reshape((*train_images_array.shape[:1], -1));
print(train_images_array.shape);

#flatten test images array;
test_images_array = test_images_array.reshape((*test_images_array.shape[:1], -1));
print(test_images_array.shape);

(1000, 784)
(1000, 784)


One output node trainning => it wouldn't actually work for we would update the same set weights, therefore the last number(image) predicted would set the weights to recognize those specific patterns. 

One output node for each of our numbers i.e. 0 to 9;

In [34]:
# weights = np.random.rand(784, 10);
# alpha = .00000000001;
# goal_pred = [];

# def goal_pred_by_label(label):
#     return {
#          0: [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
#          1: [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
#          2: [0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
#          3: [0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
#          4: [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
#          5: [0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
#          6: [0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
#          7: [0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
#          8: [0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
#          9: [0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
#     }.get(label);

# for i in range(len(images_array)):
#     goal_pred.append(goal_pred_by_label(labels_array[i]));
    
# def calc_delta(pred, labels_array):
#     goal_pred, delta = [[], []];
#     for i in range(len(labels_array)):
#         goal_pred.append(goal_pred_by_label(labels_array[i]));
#     for j in range(len(goal_pred)):
#         delta.append(np.subtract(pred[j], goal_pred[j]));
#     return delta;
        
# def neural_network(input, weights):
#     for i in range(1):
#         pred = np.dot(input, weights);
#         #print('Weight: ' + str(weights[0]));
#         msquared_error = (np.subtract(pred, goal_pred)) ** 2;
#         delta = np.subtract(pred, goal_pred);
#         weight_delta = np.dot(delta.T, input).T;
#         #print('Weight_delta: ' + str(weight_delta[0]));
#         #print('Error: ' + str(msquared_error[0]) + '\n-----------------------------');
#         weights = np.subtract(weights, (weight_delta * alpha));
#         if(i == 99999 or i == 99998 or i == 99997 or i == 99996):
#             print('Error: ' + str(msquared_error[0]) + '\n-----------------------------');
#         #plot_it_all(weights, msquared_error, weight_delta);
# #         print('Label: ' + str(labels_array[0]) + ' \nError: ' + str(msquared_error[0])
# #              + '\nPred: ' + str(pred[0]) + '\nGoal_pred: ' + str(goal_pred[0]) 
# #              + '\n-----------------------------');

# def plot_it_all(weights, errors, derivatives):
    
#     ax1.set_title('How much changing each weight' 
#                   + '\n contributed to the error?');
#     ax1.set_ylabel('Mean squared error');
#     ax1.set_xlabel('Weight');
#     ax1.scatter(weights, errors, s=None, c='g');
#     ax1.plot(weights, errors);
#     for i in range(len(weights)):
#         ax1.annotate(i, (weights[i], errors[i]));

# neural_network(images_array, weights);7

In [ ]:
MNIST stochastic gradient descent.

In [ ]:
np.random.seed(1);

alpha, iterations, hidden_size = (.001, 500, 400); 
pixels_per_image, num_of_labels = (784, 10);

synapse_0 = .2 * np.random.random((pixels_per_image, hidden_size)) - .1;
synapse_1 = .2 * np.random.random((hidden_size, num_of_labels)) - .1;

relu = lambda x:(x > 0) * x;
relu2deriv = lambda x:(x > 0);

for iteration in range(iterations):
    msquared_error_layer_2, correct_cnt = (0.0, 0);
    for index in range(len(train_images_array)):
        #forward propagation
        layer_0 = train_images_array[index:index+1];
        layer_1 = relu(layer_0.dot(synapse_0));
        #dropout
        dropout_mask = np.random.randint(2, size=layer_1.shape);
        layer_1 *= dropout_mask * 2;
        
        layer_2 = layer_1.dot(synapse_1);#l_2 = relu(l_0S_0)S_1;
        #print(layer_2);
        msquared_error_layer_2 += np.sum((layer_2 
                - train_labels_array[index:index+1]) ** 2);
        
        correct_cnt += int(np.argmax(layer_2) == np.argmax(train_labels_array[index:index+1]));
        
        layer_2_delta = layer_2 - train_labels_array[index:index+1];
        
        layer_1_delta = layer_2_delta.dot(synapse_1.T) * relu2deriv(layer_1);
        layer_1_delta *= dropout_mask;
        
        synapse_1_delta = layer_1.T.dot(layer_2_delta);
        synapse_0_delta = layer_0.T.dot(layer_1_delta);
        
        synapse_1 -= synapse_1_delta * alpha;
        synapse_0 -= synapse_0_delta * alpha;
    #time for inference    
    if(iteration % 10 == 0):
        #page -r msquared_error_layer_2
        #print(msquared_error_layer_2);
        #time for inference 
        msquared_error_test, correct_cnt_test = (0.0, 0.0);
        for index in range(len(test_images_array)):
            layer_0 = test_images_array[index:index+1];
            layer_1 = relu(layer_0.dot(synapse_0));
            layer_2 = layer_1.dot(synapse_1);
    
            msquared_error_test += np.sum((layer_2 - test_labels_array[index:index+1]) ** 2);
            correct_cnt_test += int(np.argmax(layer_2) == np.argmax(test_labels_array[index:index+1]));
            
        sys.stdout.write("\n"
                             + "I:" + str(iteration)
                             + " Train Error:" + str(msquared_error_layer_2/float(len(train_images_array)))[0:5]
                             + " Train Correct:" + str(correct_cnt/len(train_images_array))
                             + " Test Error: " + str(msquared_error_test/len(test_images_array))[0:5]
                              +" Test Correct: " + str(correct_cnt_test/len(test_images_array)));
                                 

In [ ]:
mini-bitched stochastic gradient descent.

In [ ]:
weights also can be seen as a high dimensional shape.
As you train, this shape molds around your data,
learning to distinguish one pattern from another.
The images in our testing dataset were slightly different 
than the patterns in our train set.
